In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [6]:

import zipfile

# Replace 'DATASET.zip' with the actual name of your zip file
with zipfile.ZipFile('archive (2).zip', 'r') as zip_ref:     # replace it with ur zipfilename
    zip_ref.extractall('archive (2)') # extracts to a directory called 'DATASET'

In [7]:
# Paths to dataset folders
TRAIN_DIR = '/content/archive (2)/DATASET/TRAIN'
TEST_DIR = '/content/archive (2)/DATASET/TEST'

# Image size and batch size
IMG_SIZE = (128, 128)  # Resize all images to 128x128
BATCH_SIZE = 32


train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(128,128),
    batch_size=32,
    class_mode='binary'   # keep categorical for multiclassification
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'  #keep categorical for multiclassification
)




Found 22564 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.


In [8]:
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # softmax for multi classification
    ])

    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',  #sparse_categorical_crossentropy for multiclassification
                  metrics=['accuracy'])
    return model


model = build_model()
model.summary()

#Training the Model


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:


history = model.fit(
    train_generator,
    epochs=5,
    validation_data=test_generator
)



Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


706/706 ━━━━━━━━━━━━━━━━━━━━ 37s 44ms/step - accuracy: 0.7753 - loss: 0.4921 - val_accuracy: 0.8878 - val_loss: 0.2983
Epoch 2/5
706/706 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.8342 - loss: 0.3904 - val_accuracy: 0.8774 - val_loss: 0.3145
Epoch 3/5
706/706 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.8560 - loss: 0.3451 - val_accuracy: 0.9081 - val_loss: 0.2461
Epoch 4/5
706/706 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.8687 - loss: 0.3197 - val_accuracy: 0.9021 - val_loss: 0.2608
Epoch 5/5
706/706 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.8820 - loss: 0.2930 - val_accuracy: 0.9117 - val_loss: 0.2500


In [10]:
# Saving model
model.save('recyclable_vs_organic.h5')   #potato_disease.h5

In [11]:
# 4. Evaluate the Model on Test Data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.8974 - loss: 0.2784
Test Accuracy: 0.9116593599319458


In [14]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import warnings
warnings.filterwarnings('ignore')

# Load the trained model
model = load_model('/content/recyclable_vs_organic.h5')

# Path to your custom image
image_path = r"/content/archive (2)/DATASET/TRAIN/R/R_10.jpg" # Replace with your image path

# Preprocess the image
IMG_SIZE = (128, 128)  # Must match the size used during training
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(128,128))  # Load and resize image
    img_array = img_to_array(img)  # Convert image to array
    img_array = img_array / 255.0  # Rescale to 0-1 range
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

image = preprocess_image(image_path)

# Make prediction
prediction = model.predict(image)
print("prediction prob" , prediction)
# Interpretation
if prediction[0][0] < 0.5:
    print(f"The image is predicted to be Organic:)")
else:
    print(f"The image is predicted to be: Recyclable")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 622ms/step
prediction prob [[0.9818523]]
The image is predicted to be: Recyclable


In [16]:
'''
import cv2
import numpy as np


model = tf.keras.models.load_model('recyclable_vs_organic.h5')

def preprocess_image(image):

    resized = cv2.resize(image, (128, 128))
    normalized = resized / 255.0
    return np.expand_dims(normalized, axis=0)  # Add batch dimension

def classify_trash(image):

    processed = preprocess_image(image)
    prediction = model.predict(processed)[0][0]
    if prediction[0][0] < 0.5:
      return "Organic"
    else:
      return "Recyclable"


cap = cv2.VideoCapture(1)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    label = classify_trash(frame)
    cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Trash Classifier', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
'''

error: OpenCV(4.11.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'


In [ ]:
##END OF BINARY CLASSIFICATION USING CNN

In [ ]:
# below code is for multiclassification prediction do not execute it

In [ ]:
'''
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import warnings
warnings.filterwarnings('ignore')

# Load the trained model
model = load_model('/content/potato_disease.h5')  # Updated model path

# Path to your custom image
image_path = r"/content/img.jpg"  # Replace with your image path

# Preprocess the image
IMG_SIZE = (128, 128)  # Must match the size used during training

def preprocess_image(image_path):
    img = load_img(image_path, target_size=IMG_SIZE)  # Load and resize image
    img_array = img_to_array(img)  # Convert image to array
    img_array = img_array / 255.0  # Rescale to 0-1 range
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

image = preprocess_image(image_path)

# Make prediction
prediction = model.predict(image)

# Class labels (ensure they match the model's training)
class_labels = ['Early_Blight', 'Healthy', 'Late_Blight']

# Get the predicted class index
predicted_class_index = np.argmax(prediction, axis=1)[0]
predicted_class_label = class_labels[predicted_class_index]

# Output the prediction
print(f"The image is predicted to be: {predicted_class_label}")
'''